### All Data: Consensus Genotype
Size_Sample_Deletions_AllData_6000

* Size Ranges
* Training Dataset size : 6000

In [1]:
"""
Imports
"""
import pandas as pd
import numpy as np
import graphviz
import io
from fancyimpute import KNN
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.grid_search import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import LeaveOneOut
from scipy.stats import ks_2samp
from scipy import stats
from matplotlib import pyplot
from sklearn import preprocessing
from scipy.linalg import svd
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA as sklearnPCA
import plotly.plotly as py
from sklearn.cluster import DBSCAN
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import f1_score, precision_score
from sklearn import preprocessing
from ggplot import *
from bokeh.charts import TimeSeries
from bokeh.models import HoverTool
from bokeh.plotting import show
from bokeh.charts import Scatter, Histogram, output_file, show
from bokeh.plotting import figure, show, output_file, ColumnDataSource
from bokeh.io import output_notebook
from bokeh.charts import Bar, output_file, show
import bokeh.palettes as palettes
from bokeh.models import HoverTool, BoxSelectTool, Legend
from sklearn import (manifold, datasets, decomposition, ensemble,
                     discriminant_analysis, random_projection)

/Users/lmc2/anaconda/envs/NIHFAES/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/lmc2/anaconda/envs/NIHFAES/lib/python3.5/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


***
Training Set

Count : 6000
*** 

In [2]:
# Import Training Data
df_train = pd.read_csv('/Volumes/lesleydata/size_sample/Step4_ML/dataframes/Step1_CombinedDFs/5k_testSet/train/6k.DEL.test.csv')
df_train_2 = pd.read_csv('/Volumes/lesleydata/size_sample/Step4_ML/dataframes/Step1_CombinedDFs/5k_testSet/train/6k.DEL.test.csv')
df_train.rename(columns={'size': 'Size'}, inplace=True)
df_train.shape

FileNotFoundError: File b'/Volumes/lesleydata/size_sample/Step4_ML/dataframes/Step1_CombinedDFs/5k_testSet/train/6k.DEL.test.csv' does not exist

In [ ]:
train_set = pd.DataFrame()
train_set = df_train_2

In [ ]:
train_set['GTcons'].replace(0, 'Homozygous_Reference', inplace=True)
train_set['GTcons'].replace(1, 'Heterozygous_Variant', inplace=True)
train_set['GTcons'].replace(2, 'Homozygous_Variant', inplace=True)

** Count of Labels in Training Set **

<a id='imbalance'></a>

In [ ]:
pd.value_counts(train_set['GTcons'].values, sort=False)

In [ ]:
### Drop columns that are not shared by both dataframes
df_train.drop(['Ill300x.GT'], axis=1, inplace = True)
df_train.drop(['Ill250.GT'], axis=1, inplace = True)
df_train.drop(['IllMP.GT'], axis=1, inplace = True)
df_train.drop(['TenX.GT'], axis=1, inplace = True)
df_train.drop(['pacbio.GT'], axis=1, inplace = True)
df_train.drop(['GTconflict'], axis=1, inplace = True)
df_train.drop(['GTsupp'], axis=1, inplace = True)
df_train.drop(['sample'], axis=1, inplace = True)
df_train.drop(['SVtype'], axis=1, inplace = True)
df_train.drop(['type'], axis=1, inplace = True)
df_train.drop(['id'], axis=1, inplace = True)
df_train.drop(['New_ID'], axis=1, inplace = True)

In [ ]:
df_train['chrom'].replace('X', 23, inplace=True)
df_train['chrom'].replace('Y', 24, inplace=True)

***
Test Set

Size separated
***

<a id='hom_ref'></a>

In [ ]:
# Import Test Data
# SVanalyzer generated training data
df_test_size = pd.read_csv('/Volumes/lesleydata/size_sample/Step4_ML/dataframes/Step1_CombinedDFs/new_sizeSample_testSet/DEL/All_size_sample_TestSet.csv')
df_test_size_2 = pd.read_csv('/Volumes/lesleydata/size_sample/Step4_ML/dataframes/Step1_CombinedDFs/new_sizeSample_testSet/DEL/All_size_sample_TestSet.csv')
df_test_size.rename(columns={'size': 'Size'}, inplace=True)
df_test_size.shape

In [ ]:
### Drop columns that are not shared by both dataframes
df_test_size.drop(['Ill300x.GT'], axis=1, inplace = True)
df_test_size.drop(['Ill250.GT'], axis=1, inplace = True)
df_test_size.drop(['IllMP.GT'], axis=1, inplace = True)
df_test_size.drop(['TenX.GT'], axis=1, inplace = True)
df_test_size.drop(['pacbio.GT'], axis=1, inplace = True)
df_test_size.drop(['GTconflict'], axis=1, inplace = True)
df_test_size.drop(['GTsupp'], axis=1, inplace = True)
df_test_size.drop(['sample'], axis=1, inplace = True)
df_test_size.drop(['SVtype'], axis=1, inplace = True)
df_test_size.drop(['type'], axis=1, inplace = True)
df_test_size.drop(['id'], axis=1, inplace = True)
df_test_size.drop(['New_ID'], axis=1, inplace = True)

In [ ]:
df_test_size.head(1)

In [ ]:
df_test_size['chrom'].replace('X', 23, inplace=True)
df_test_size['chrom'].replace('Y', 24, inplace=True)

***
Impute missing values using KNN
***

In [ ]:
# Store training data in a new variable which will be converted to a matrix
X = df_train
X.head(3)

In [ ]:
# Convert dataframe to matrix
X=X.as_matrix()

#Imput missing values from three closest observations
X_imputed=KNN(k=3).complete(X)
X=pd.DataFrame(X_imputed)

In [ ]:
# Store header values in a list, will be used later to re-label the matrix post KNN imputation
dftrain_header = list(df_train.columns.values)
X.columns = dftrain_header
X.head(3)

In [ ]:
# Store Labels in a new 'Y' DataFrame
Y = pd.DataFrame()
Y = X['GTcons']

In [ ]:
#Count the number of labels
pd.value_counts(Y.values, sort=False)

In [ ]:
# Remove labels from feature set
X.drop(['GTcons'],axis=1, inplace = True)

In [ ]:
# Order features
X4 = X.reindex_axis(sorted(X.columns), axis=1)

***
Machine Learning
***

<a id='machine_learning'></a>

** Train Random Forest Classifier **

<a id='multi_run'></a>

** Determine Number of trees: Out of Bag Error **

In [ ]:
# Train Test Split
# Train on 70% of the data and test on 30%
X_train, X_test, y_train, y_test = train_test_split(X4, Y, test_size=0.3)

In [ ]:
# Train Random Forest Classifier
model = RandomForestClassifier(oob_score=True) 
model.fit(X_train, y_train)

In [ ]:
print('The OOB prediction of accuracy is: {oob}%'.format(oob=model.oob_score_ * 100))

In [ ]:
sns.set_style("white")

In [ ]:
n_estimators = 150
trees = []
oob = []
for i in range(1, n_estimators):
    model.set_params(n_estimators=i)
    model.fit(X_train, y_train)
    trees += [i]
    oob += [model.oob_score_] 
    
    
df_oob = pd.DataFrame()
df_oob['trees'] = trees
df_oob['oob'] = oob


plt.plot(trees, oob)

plt.xlabel("Number of Trees [n_estimators]")
plt.ylabel("OOB error rate")
plt.legend(loc="upper right")
plt.ylim(0, 1.0)
plt.savefig('/Volumes/lesleydata/size_sample/Step4_ML/plots/DEL/all_data/5k_Test_Set/6000_train/del_alldata_6000_trees_oob.png', bbox_inches='tight')
plt.show()

** Train Model Using Optimal Tuning Parameters**

In [ ]:
model = RandomForestClassifier(n_estimators=20, random_state=4, class_weight="balanced") 
model.fit(X_train, y_train)

In [ ]:
feature_importances = pd.Series(model.feature_importances_, index=X_train.columns)

In [ ]:
pred = model.predict(X_test)

In [ ]:
print('Precision score of the training subset: {:.3f}'.format(precision_score(y_test, pred, average='micro'))) 
from sklearn.metrics import accuracy_score
print('Accuracy score of the training subset: {:.3f}'.format(accuracy_score(y_test, pred))) 

In [ ]:
# Add original labels and predicted labels back to the original dataframe
df_Xtest = pd.DataFrame(X_test)
labels = pd.DataFrame(y_test)

In [ ]:
df_Xtest['predicted_label'] = pred
df_Xtest['GTcons'] = df_train['GTcons']
df_Xtest['chrom'] = df_train['chrom']
df_Xtest['start'] = df_train['start']
df_Xtest['end'] = df_train['end']

In [ ]:
df_Xtest['GTcons'].replace(0.0, 'Homozygous_Reference', inplace=True)
df_Xtest['GTcons'].replace(1.0, 'Heterozygous_Variant', inplace=True)
df_Xtest['GTcons'].replace(2.0, 'Homozygous_Variant', inplace=True)
df_Xtest['predicted_label'].replace(0.0, 'Homozygous_Reference', inplace=True)
df_Xtest['predicted_label'].replace(1.0, 'Heterozygous_Variant', inplace=True)
df_Xtest['predicted_label'].replace(2.0, 'Homozygous_Variant', inplace=True)

In [ ]:
pd.value_counts(df_Xtest['GTcons'].values, sort=False)

In [ ]:
pd.value_counts(df_Xtest['predicted_label'].values, sort=False)

In [ ]:
from sklearn.metrics import confusion_matrix
ytest = df_Xtest['GTcons']
predict = df_Xtest['predicted_label']
print(confusion_matrix(ytest, predict))

In [ ]:
pd.crosstab(ytest, predict, rownames=['True'], colnames=['Predicted'], margins=True)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(ytest, predict))

***
Predict

Predict labels for test set
***

** Load Data **

In [ ]:
df_test_size.drop(['GTcons'],axis=1, inplace=True)
X2 = df_test_size

** Impute missing values using KNN **

In [ ]:
#Convert dataframe to matrix
X2=X2.as_matrix()
X2=pd.DataFrame(X2)

# Imput missing values from three closest observations
X2_imputed=KNN(k=3).complete(X2)
X2=pd.DataFrame(X2_imputed)

In [ ]:
df_test_header = list(df_test_size.columns.values)
X2.columns = df_test_header
X2.head(3)

In [ ]:
X3 = pd.DataFrame()
X3 = X2
X3.head(3)

In [ ]:
# Order features
X5 = X2.reindex_axis(sorted(X2.columns), axis=1)

In [ ]:
pred = model.predict(X5)
pred_prob = model.predict_proba(X5)
pred_prob_log = model.predict_log_proba(X5)

In [ ]:
X5['predicted_label'] = pred
X5['chrom'] = df_test_size_2['chrom']
X5['GTcons'] = df_test_size_2['GTcons']
X5['start'] = df_test_size_2['start']
X5['end'] = df_test_size_2['end']
X5['Size'] = df_test_size_2['Size']
X5['GTsupp'] = df_test_size_2['GTsupp']

In [ ]:
X6 = pd.concat([X5, pd.DataFrame(pred_prob, columns=['1','2','3'])])
X7 = pd.concat([X5, pd.DataFrame(pred_prob_log, columns=['1','2','3'])])
X6.to_csv('/Volumes/lesleydata/size_sample/Step4_ML/plots/DEL/all_data/5k_Test_Set/size_samp/df_size_sampTest_df1.csv', index=False)

In [ ]:
X6 = pd.read_csv('/Volumes/lesleydata/size_sample/Step4_ML/plots/DEL/all_data/5k_Test_Set/size_samp/df_size_sampTest_df1.csv')

In [ ]:
X6.rename(columns={'1': 'Homozygous_Reference_GTcons'}, inplace=True)
X6.rename(columns={'2': 'Heterozygous_Variant_GTcons'}, inplace=True)
X6.rename(columns={'3': 'Homozygous_Variant_GTcons'}, inplace=True)
X6.rename(columns={'predicted_label': 'predicted_GTcons_label'}, inplace=True)

In [ ]:
X6.to_csv('/Volumes/lesleydata/size_sample/Step4_ML/plots/DEL/all_data/5k_Test_Set/size_samp/df_size_sampTest_df2.csv', index=False)

In [ ]:
X6['GTcons'].dtypes

In [ ]:
# Drop NaN Labels
X6 = X6[np.isfinite(X6['GTcons'])]
X6 = X6[np.isfinite(X6['predicted_GTcons_label'])]

***
Label Analysis
***

In [ ]:
from sklearn.metrics import confusion_matrix
consensus_GT = X6['GTcons']
predict = X6['predicted_GTcons_label']
print(confusion_matrix(consensus_GT, predict))

In [ ]:
X6['GTcons'].replace(0, 'Homozygous_Reference', inplace=True)
X6['GTcons'].replace(1, 'Heterozygous_Variant', inplace=True)
X6['GTcons'].replace(2, 'Homozygous_Variant', inplace=True)
X6['predicted_GTcons_label'].replace(0.0, 'Homozygous_Reference', inplace=True)
X6['predicted_GTcons_label'].replace(1.0, 'Heterozygous_Variant', inplace=True)
X6['predicted_GTcons_label'].replace(2.0, 'Homozygous_Variant', inplace=True)

In [ ]:
print('Precision score of the prediction subset: {:.3f}'.format(precision_score(consensus_GT, predict, average='micro'))) 
print('Accuracy score of the prediction subset: {:.3f}'.format(accuracy_score(consensus_GT, predict))) 

In [ ]:
pd.crosstab(consensus_GT, predict, rownames=['True'], colnames=['Predicted'], margins=True)

In [ ]:
conf_mat = pd.crosstab(consensus_GT, predict, rownames=['True'], colnames=['Predicted'], margins=True)
conf_mat.to_csv('/Volumes/lesleydata/size_sample/Step4_ML/plots/DEL/all_data/5k_Test_Set/size_samp/Test_SizeSamp_All_confMatrix.csv')

** Labels with Pred_Prob >=0.9 **

In [ ]:
high_conf_labels = X6[(X6['Homozygous_Reference_GTcons'] == 1) | (X6['Homozygous_Reference_GTcons'] >= 0.9) | (X6['Heterozygous_Variant_GTcons'] == 1) | (X6['Heterozygous_Variant_GTcons'] >= 0.9) | (X6['Homozygous_Variant_GTcons'] == 1) | (X6['Homozygous_Variant_GTcons'] >= 0.9)]

In [ ]:
consensus_GT = high_conf_labels['GTcons']
predict = high_conf_labels['predicted_GTcons_label']
pd.crosstab(consensus_GT, predict, rownames=['True'], colnames=['Predicted'], margins=True)

In [ ]:
consensus_GT = high_conf_labels['GTcons']
predict = high_conf_labels['predicted_GTcons_label']
conf_mat = pd.crosstab(consensus_GT, predict, rownames=['True'], colnames=['Predicted'], margins=True)
conf_mat.to_csv('/Volumes/lesleydata/size_sample/Step4_ML/plots/DEL/all_data/5k_Test_Set/size_samp/Test_SizeSamp_All_confMatrix_hi.csv')

In [ ]:
print(classification_report(consensus_GT, predict))

***
Size Separate Label Analysis
***

In [ ]:
#Size Bins
bins = [0, 50, 100,300,400,1000,5999,45516]
X6['Size'] = X6['Size'].abs()
group_names_size = ['0-50', '50-100', '100-300', '300-400', '400-1000', '1000-5999', '6000+']
X6['size_bin'] = pd.cut(X6['Size'], bins, labels=group_names_size)

#Separate dataframes based on size_bin
df_20to50 = X6[X6['size_bin'].str.contains('0-50')]
df_50to100 = X6[X6['size_bin'].str.contains('50-100')]
df_100to300 = X6[X6['size_bin'].str.contains('100-300')]
df_300to400 = X6[X6['size_bin'].str.contains('300-400')]
df_400to1000 = X6[X6['size_bin'].str.contains('400-1000')]
df_1000to6000 = X6[X6['size_bin'].str.contains('1000-5999')]
df_6000 = X6[X6['size_bin'].str.contains('6000+')]

In [ ]:
#Confusion Matrices
consensus_GT = df_20to50['GTcons']
predict = df_20to50['predicted_GTcons_label']
pd.crosstab(consensus_GT, predict, rownames=['True'], colnames=['Predicted'], margins=True)

In [ ]:
consensus_GT = df_50to100['GTcons']
predict = df_50to100['predicted_GTcons_label']
print('df_50to100')
pd.crosstab(consensus_GT, predict, rownames=['True'], colnames=['Predicted'], margins=True)

In [ ]:
consensus_GT = df_100to300['GTcons']
predict = df_100to300['predicted_GTcons_label']
print('df_100to300')
pd.crosstab(consensus_GT, predict, rownames=['True'], colnames=['Predicted'], margins=True)

In [ ]:
consensus_GT = df_300to400['GTcons']
predict = df_300to400['predicted_GTcons_label']
print('df_300to400')
pd.crosstab(consensus_GT, predict, rownames=['True'], colnames=['Predicted'], margins=True)

In [ ]:
consensus_GT = df_400to1000['GTcons']
predict = df_400to1000['predicted_GTcons_label']
print('df_400to1000')
pd.crosstab(consensus_GT, predict, rownames=['True'], colnames=['Predicted'], margins=True)

In [ ]:
consensus_GT = df_1000to6000['GTcons']
predict = df_1000to6000['predicted_GTcons_label']
print('df_1000to6000')
pd.crosstab(consensus_GT, predict, rownames=['True'], colnames=['Predicted'], margins=True)

In [ ]:
consensus_GT = df_6000['GTcons']
predict = df_6000['predicted_GTcons_label']
print('df_6000')
pd.crosstab(consensus_GT, predict, rownames=['True'], colnames=['Predicted'], margins=True)

In [ ]:
#Calculate average pred prob for each group
df_20to50_homVar = df_20to50[df_20to50['GTcons'] == 'Homozygous_Variant']['Homozygous_Variant_GTcons']
df_20to50_hetVar = df_20to50[df_20to50['GTcons'] == 'Heterozygous_Variant']['Heterozygous_Variant_GTcons']
df_20to50_homRef = df_20to50[df_20to50['GTcons'] == 'Homozygous_Reference']['Homozygous_Reference_GTcons']

df_20to50_homVar.mean()
df_20to50_hetVar.mean()
df_20to50_homRef.mean()
print('20 to 50 bp Hom Var {}'.format(df_20to50_homVar.mean())) 
print('20 to 50 bp Het Var {}'.format(df_20to50_hetVar.mean())) 
print('20 to 50 bp Hom Ref {}'.format(df_20to50_homRef.mean())) 

df_50to100_homVar = df_50to100[df_50to100['GTcons'] == 'Homozygous_Variant']['Homozygous_Variant_GTcons']
df_50to100_hetVar = df_50to100[df_50to100['GTcons'] == 'Heterozygous_Variant']['Heterozygous_Variant_GTcons']
df_50to100_homRef = df_50to100[df_50to100['GTcons'] == 'Homozygous_Reference']['Homozygous_Reference_GTcons']

df_50to100_homVar.mean()
df_50to100_hetVar.mean()
df_50to100_homRef.mean()
print('50to100 bp Hom Var {}'.format(df_50to100_homVar.mean())) 
print('50to100 bp Het Var {}'.format(df_50to100_hetVar.mean())) 
print('50to100 bp Hom Ref {}'.format(df_50to100_homRef.mean())) 

df_100to300_homVar = df_100to300[df_100to300['GTcons'] == 'Homozygous_Variant']['Homozygous_Variant_GTcons']
df_100to300_hetVar = df_100to300[df_100to300['GTcons'] == 'Heterozygous_Variant']['Heterozygous_Variant_GTcons']
df_100to300_homRef = df_100to300[df_100to300['GTcons'] == 'Homozygous_Reference']['Homozygous_Reference_GTcons']

df_100to300_homVar.mean()
df_100to300_hetVar.mean()
df_100to300_homRef.mean()
print('100to300 bp Hom Var {}'.format(df_100to300_homVar.mean())) 
print('100to300 bp Het Var {}'.format(df_100to300_hetVar.mean())) 
print('100to300 bp Hom Ref {}'.format(df_100to300_homRef.mean())) 

df_300to400_homVar = df_300to400[df_300to400['GTcons'] == 'Homozygous_Variant']['Homozygous_Variant_GTcons']
df_300to400_hetVar = df_300to400[df_300to400['GTcons'] == 'Heterozygous_Variant']['Heterozygous_Variant_GTcons']
df_300to400_homRef = df_300to400[df_300to400['GTcons'] == 'Homozygous_Reference']['Homozygous_Reference_GTcons']

df_300to400_homVar.mean()
df_300to400_hetVar.mean()
df_300to400_homRef.mean()
print('300to400 bp Hom Var {}'.format(df_300to400_homVar.mean())) 
print('300to400 bp Het Var {}'.format(df_300to400_hetVar.mean())) 
print('300to400 bp Hom Ref {}'.format(df_300to400_homRef.mean())) 

df_400to1000_homVar = df_400to1000[df_400to1000['GTcons'] == 'Homozygous_Variant']['Homozygous_Variant_GTcons']
df_400to1000_hetVar = df_400to1000[df_400to1000['GTcons'] == 'Heterozygous_Variant']['Heterozygous_Variant_GTcons']
df_400to1000_homRef = df_400to1000[df_400to1000['GTcons'] == 'Homozygous_Reference']['Homozygous_Reference_GTcons']

df_400to1000_homVar.mean()
df_400to1000_hetVar.mean()
df_400to1000_homRef.mean()
print('400to1000 bp Hom Var {}'.format(df_400to1000_homVar.mean())) 
print('400to1000 bp Het Var {}'.format(df_400to1000_hetVar.mean())) 
print('400to1000 bp Hom Ref {}'.format(df_400to1000_homRef.mean())) 

df_1000to6000_homVar = df_1000to6000[df_1000to6000['GTcons'] == 'Homozygous_Variant']['Homozygous_Variant_GTcons']
df_1000to6000_hetVar = df_1000to6000[df_1000to6000['GTcons'] == 'Heterozygous_Variant']['Heterozygous_Variant_GTcons']
df_1000to6000_homRef = df_1000to6000[df_1000to6000['GTcons'] == 'Homozygous_Reference']['Homozygous_Reference_GTcons']

df_1000to6000_homVar.mean()
df_1000to6000_hetVar.mean()
df_1000to6000_homRef.mean()
print('1000to6000 bp Hom Var {}'.format(df_1000to6000_homVar.mean())) 
print('1000to6000 bp Het Var {}'.format(df_1000to6000_hetVar.mean())) 
print('1000to6000 bp Hom Ref {}'.format(df_1000to6000_homRef.mean())) 

df_6000_homVar = df_6000[df_6000['GTcons'] == 'Homozygous_Variant']['Homozygous_Variant_GTcons']
df_6000_hetVar = df_6000[df_6000['GTcons'] == 'Heterozygous_Variant']['Heterozygous_Variant_GTcons']
df_6000_homRef = df_6000[df_6000['GTcons'] == 'Homozygous_Reference']['Homozygous_Reference_GTcons']

df_6000_homVar.mean()
df_6000_hetVar.mean()
df_6000_homRef.mean()
print('6000 bp Hom Var {}'.format(df_6000_homVar.mean())) 
print('6000 bp Het Var {}'.format(df_6000_hetVar.mean())) 
print('6000 bp Hom Ref {}'.format(df_6000_homRef.mean())) 

In [ ]:
df_20to50_homVar.mean()